# LIMIT ORDER BOOK RECONSTRUCTION

### LOADING LIBRARIES

In [ ]:
# Importing required libraries
import pandas as pd
import numpy as np
import json
import h5py

# Import time functions
from time import time
from time import ctime
from datetime import datetime

### SETTING SOME VARIABLES

In [ ]:
N = 10 # Depth (number of levels)
date = open('date.txt').read()
hour = 1

### LOADING DATA

In [ ]:
# Loading events csv file
Message_Book = pd.read_csv('Raw Kaiko Dataset/Events/kaiko-obl3_l3-event-coinbase_' + date + '_daily_coinbase_btc-usd_' + date + '.csv.gz',
                           delimiter=';', 
                           usecols=[1], 
                           names=['Events'], 
                           header=None,
                           compression='gzip')

In [ ]:
# Loading snapshot csv file
Snapshots = pd.read_csv('Raw Kaiko Dataset/Snapshots/kaiko-obl3_l3-snapshot-coinbase_' + date + '_daily_coinbase_btc-usd_' + date + '.csv.gz',
                      delimiter=';',
                      usecols=[1],
                      names=['LOB_Snapshot'],
                      header=None,
                      compression='gzip')

### INITIAL LIMIT ORDER BOOK STATE

In [ ]:
# Loading initial LOB State, bids, asks and sequence
full_order_book = json.loads(Snapshots['LOB_Snapshot'][hour])
asks = np.array(full_order_book['asks'])
bids = np.array(full_order_book['bids'])
snapshot_sequence_number = full_order_book['sequence']
next_sequence_number = json.loads(Snapshots['LOB_Snapshot'][hour + 1])['sequence']

### DEFINING START AND END POINTS

In [ ]:
# Getting the first event sequence to serve as point of referece
first_event_sequence = json.loads(Message_Book['Events'][0])['sequence']

# Getting the indexes
start_index = snapshot_sequence_number - first_event_sequence
end_index = start_index + (next_sequence_number - snapshot_sequence_number)

# Sequence numbers base on the start and end indexes
try:
    start_sequence = json.loads(Message_Book['Events'][start_index])['sequence']
except KeyError:
    start_sequence = 0
    
try:    
    end_sequence = json.loads(Message_Book['Events'][end_index])['sequence']
except KeyError:
    end_sequence = 0

### GET INDEX

In [ ]:
def get_index(keyword):
    keyword = str(keyword) # Making sure that the keyword is a string
    
    for index, message in enumerate(Message_Book['Events']):
        if keyword in message:
            return index
    
    return 'Not found'

### ASSESMENT OF THE DATA (Duplicated events, missing events )

In [ ]:
# Setting the initial status of the data as OK
data_status = 'OK'

# Getting starting index
if start_sequence != snapshot_sequence_number:
    
    start_index = get_index(snapshot_sequence_number)
    
    if start_index == 'Not found':
        data_status = 'Missing'
        print('WARNING: Initial sequence number not found, do not continue')
        # Writing a report
        with open(f'LOB Reconstructed CUT-Details/LOB_{date}_{hour}_{data_status}.txt','w+') as f:
            f.write(str(next_sequence_number - snapshot_sequence_number))
            f.close()
    
    else:
        start_sequence = json.loads(Message_Book['Events'][start_index])['sequence']
        end_index = start_index + (next_sequence_number - snapshot_sequence_number)
        end_sequence = json.loads(Message_Book['Events'][end_index])['sequence']
        
if data_status != 'Missing':
        
    # Getting ending index
    if end_sequence != next_sequence_number:
        end_index = get_index(next_sequence_number)

        if end_index == 'Not found':
            print('WARNING: End sequence number not found')
            end_index = Message_Book['Events'].shape[0] - 1
            end_sequence = json.loads(Message_Book['Events'][end_index])['sequence']

        else:
            end_sequence = json.loads(Message_Book['Events'][end_index])['sequence']


    # Removing duplicate values from subset 'messages_data'
    messages_data = Message_Book[start_index + 1: end_index + 1]
    # Resetting indexes to initialize in zero
    messages_data = messages_data.reset_index(drop=True)

    original_messages = messages_data.shape[0]
    messages_data = messages_data.drop_duplicates()
    no_duplicated_messages = messages_data.shape[0]

    print('A total of: ', original_messages - no_duplicated_messages, 'events were duplicated')
    
    
    # Verifying sequential numbers
    temp_sequence = start_sequence

    for index, message in enumerate(messages_data['Events']):
        event = json.loads(message)
        event_sequence = event['sequence']

        if event_sequence == temp_sequence + 1:
            temp_sequence = event_sequence
        else:
            end_index = start_index + index
            # Truncate the messages_data
            messages_data = messages_data[:index]
            # Change the data status
            data_status = 'CUT'
            print('Index: ', index, ' Difference: ', event_sequence - temp_sequence)
            print('Dataset truncated from a total size of: ',  next_sequence_number - snapshot_sequence_number,
                 'to a size of: ', index)
            # Writing a report
            with open(f'LOB Reconstructed CUT-Details/LOB_{date}_{hour}_{data_status}.txt','w+') as f:
                f.write(str(next_sequence_number - snapshot_sequence_number) + '\n') # Original number of events
                f.write(str(index) + '\n') # Number of events after cut
                f.write(str(next_sequence_number - snapshot_sequence_number - index) + '\n') # Number of events not processed
                f.write(str(event_sequence - temp_sequence)) # Number of missing events (first ocurrence found)
                f.close()
            break

## DEFINING FUNCTIONS TO BE USED

### Fill LOB Levels (Ask and Bid)

In [ ]:
def fill_levels(side, depth):
    temp_array = np.zeros((depth, 2))
    level = 0 # Price level
    
    temp_array_participants = np.zeros((depth, 1))
    n = 0 # Number of participants per price level
    
    if side == 'buy':
        prev_price = float(bids[0][0])
        temp_array[0][0] = prev_price
        for order in bids:
            if level <= depth - 1:
                level_price = float(order[0])
                if level_price < prev_price:
                    prev_price = level_price
                    level += 1
                    if level < depth:
                        temp_array[level][0] = level_price
                        temp_array[level][1] = float(order[1])
                        temp_array_participants[level] = 1
                else:
                    temp_array[level][1] += float(order[1])
                    temp_array_participants[level] += 1
            else:
                break
                
    elif side == 'sell':
        prev_price = float(asks[0][0])
        temp_array[0][0] = prev_price
        for order in asks:
            if level <= depth - 1:
                level_price = float(order[0])
                if level_price > prev_price:
                    prev_price = level_price
                    level += 1
                    if level < depth:
                        temp_array[level][0] = level_price
                        temp_array[level][1] = float(order[1])
                        temp_array_participants[level] = 1
                else:
                    temp_array[level][1] += float(order[1])
                    temp_array_participants[level] += 1
            else:
                break
                
    return temp_array, temp_array_participants

### Update the first N levels

In [ ]:
def update_x_t(side, depth, x_t, n_t):
    # Add update to its respective side
    if side == 'buy':
        x_t[:, 2:], n_t[:, 1:] = fill_levels(side, depth)
    else:
        x_t[:, :2], n_t[:, :1] = fill_levels(side, depth)

    return x_t, n_t

### Add order to Limit Order Book

In [ ]:
def add_order_to_orderbook(order_id, price, remaining_size, side, book_side):
    
    if side == 'buy':
        
        # Check if price level is already in the order book
        indexes = np.where(book_side[:, 0] == price)
        
        if indexes[0].shape[0] == 0:# There is no price level with the same price
            price_temp = float(price)
            counter = 0
            for row in book_side:
                if float(row[0]) < price_temp:
                    updated_side = np.insert(book_side, counter, [price, remaining_size, order_id], axis=0)
                    break
                else:
                    counter += 1
            else: # It is added at the end of the list, no price level smaller than this
                updated_side = np.insert(book_side, counter, [price, remaining_size, order_id], axis=0)

        else: # The price level is alredy in the order book
            last_index = indexes[0][-1]
            updated_side = np.insert(book_side, last_index + 1, [price, remaining_size, order_id], axis = 0)
            

    elif side == 'sell':
        # Check if price level is already in the order book
        indexes = np.where(book_side[:, 0] == price)
        
        if indexes[0].shape[0] == 0:# There is no price level with the same price
            price_temp = float(price)
            counter = 0
            for row in book_side:
                if float(row[0]) > price_temp:
                    updated_side = np.insert(book_side, counter, [price, remaining_size, order_id], axis=0)
                    break
                else:
                    counter += 1
            else: # It is added at the end of the list, no price level greater than this
                updated_side = np.insert(book_side, counter, [price, remaining_size, order_id], axis=0)

        else: # The price level is alredy in the order book
            last_index = indexes[0][-1]
            updated_side = np.insert(book_side, last_index + 1, [price, remaining_size, order_id], axis = 0)
    
    return updated_side

### Remove from Limit Order Book

In [ ]:
def remove_from_orderbook(side, order_id, book_side):
       
    if side == 'buy':
        updated_side = book_side[book_side[:, 2] != order_id]
                                                             
    elif side == 'sell':
        updated_side = book_side[book_side[:, 2] != order_id]
                                                              
    return updated_side

### Subtract order size

In [ ]:
def subtract_from_orderbook(maker_order_id, size, side, book_side): 
    
    if side == 'buy':
        
        index = np.where(book_side[:, 2] == maker_order_id)[0][0]
        actual_size = book_side[index][1]
        new_size = round(float(actual_size) - float(size), 8)
        # Update the new value
        book_side[index][1] = '{:.8f}'.format(new_size).rstrip('0')
        
        
    elif side == 'sell':

        index = np.where(book_side[:, 2] == maker_order_id)[0][0]
        actual_size = book_side[index][1]
        new_size = round(float(actual_size) - float(size), 8)
        # Update the new value
        book_side[index][1] = '{:.8f}'.format(new_size).rstrip('0') 

    return book_side

### Change order size

In [ ]:
def change_order_size(order_id, size, side, book_side):  
    
    if side == 'buy':
        
        index = np.where(book_side[:, 2] == order_id)[0][0]
        # Update the new value
        book_side[index][1] = size        
        
    elif side == 'sell':

        index = np.where(book_side[:, 2] == order_id)[0][0]
        # Update the new value
        book_side[index][1] = size
    
    return book_side

### Get price

In [ ]:
# Getting the price before removing the order, some events does not come with a price key.
def get_price_done(side, event, book_side):
    
    try:
        price = float(event['price'])
        
    except KeyError:

        if side == 'buy':
            index = np.where(book_side[:, 2] == order_id)[0][0]
            price = float(book_side[index][0])

        elif side == 'sell':
            index = np.where(book_side[:, 2] == order_id)[0][0]
            price = float(book_side[index][0])
    
    return price

### Verify bids and asks

In [ ]:
def checker():
    full_order_book_next = json.loads(Snapshots['LOB_Snapshot'][hour + 1])
    asks_next = np.array(full_order_book_next['asks'])
    bids_next = np.array(full_order_book_next['bids'])
    
    for i in range(asks.shape[0]):
        if not np.all(asks[i] == asks_next[i]):
            print('ASKS mismatch at: ', i)
            print(asks[i])
            print(asks_next[i])

    for i in range(bids.shape[0]):
        if not np.all(bids[i] == bids_next[i]):
            print('BIDS mismatch at: ', i)
            print(bids[i])
            print(bids_next[i])

### ISO 8601 to time epoch

In [ ]:
# Function to convert ISO 8601 Formatted date time to epoch
def from_iso_to_epoch(date_time):
    utc_time = datetime.strptime(date_time, "%Y-%m-%dT%H:%M:%S.%fZ")
    epoch_time = (utc_time - datetime(1970, 1, 1)).total_seconds()
    return epoch_time

### Extended features

In [ ]:
def extend_features(counter, past_event_time, n_t, event_type, side, event_time, price=None, size=None, reason=None, 
                    continuous=False, both_sides=False, unopened=False):
    
    extended_features_t = np.zeros((1,13))
    
    # Fill side
    if not both_sides:
        if side == 'buy':
            extended_features_t[0, 0] = 1 
        elif side == 'sell':
            extended_features_t[0, 1] = 1
    else:
        extended_features_t[0, 2] = 1 # Both sides
    
    # Fill order-event type
    if event_type == 'open':
        extended_features_t[0, 3] = 1
        
    elif event_type == 'done':
        if reason == 'canceled':
            extended_features_t[0, 4] = 1
        elif reason == 'market_filled':
            extended_features_t[0, 5] = 1
        elif reason == 'limit_filled':
            extended_features_t[0, 6] = 1
    
    elif event_type == 'change':
        extended_features_t[0, 7] = 1
        
        
    # Fill price and volume(size)
    if continuous:
        
        if side == 'buy':

            if float(price) < N_T[counter, 28]:
                if N_T[counter, 28] != 0:
                    price = N_T[counter, 28]              
                    size = N_T[counter, 29]

            extended_features_t[0, 8] = float(price) # Best bid
            extended_features_t[0, 9] = float(size)  # Best bid size
            extended_features_t[0, 10] = N_T[counter, 30] # We keep the same value it already has, 0 or something
            extended_features_t[0, 11] = N_T[counter, 31] # We keep the same value it already has, 0 or something



        elif side == 'sell':
            if float(price) > N_T[counter, 30]:
                if N_T[counter , 30] != 0:
                    price = N_T[counter, 30]
                    size = N_T[counter, 31]

            extended_features_t[0, 8] = N_T[counter, 28] # We keep the same value it already has, 0 or something
            extended_features_t[0, 9] = N_T[counter, 29] # We keep the same value it already has, 0 or something                 
            extended_features_t[0, 10] = float(price) # Best ask
            extended_features_t[0, 11] = float(size) # Best ask size
    
    else:
        if unopened: # Information for price or size is recorded for market or limit orders that got filled.
            if side == 'buy':
                extended_features_t[0, 8] = aggregated_multiplication / aggregated_size
                extended_features_t[0, 9] = aggregated_size

            elif side == 'sell':
                extended_features_t[0, 10] = aggregated_multiplication / aggregated_size
                extended_features_t[0, 11] = aggregated_size
        
        else:
            if side == 'buy':        
                extended_features_t[0, 8] = float(price)
                extended_features_t[0, 9] = float(size)

            elif side == 'sell':
                extended_features_t[0, 10] = float(price)
                extended_features_t[0, 11] = float(size)

    
    # Fill elapsed seconds since last event
    if continuous:
        extended_features_t[0, 12] = N_T[counter, 32]
    else:
        event_time_decimal = from_iso_to_epoch(event_time)
        past_event_time_decimal = from_iso_to_epoch(past_event_time)
        extended_features_t[0, 12] = round(event_time_decimal - past_event_time_decimal, 6)
    
    # Passing past_event_time
    past_event_time = event_time     
    
    # Passing past_event_side
    if both_sides:
        past_event_side = 'both'
    else:
        past_event_side = side
    
    #Passing past_event_type
    past_event_type = event_type
    
    # Concatenate features to form a vector of shape=(1,33)
    extended_features_t = np.concatenate((n_t.reshape((1, 20)), extended_features_t), axis=1)
    
    return extended_features_t, past_event_time, past_event_side, past_event_type

### Aggregate events

In [ ]:
def aggregate_events(X_T, N_T, counter, past_event_time, past_event_side, past_event_type, n_t, event_type, side, event_time, price=None, size=None, reason=None):

    both_sides = False # Initialize both_sides as false (same side of orders)

    if event_type == 'done':
    
        if market_orders:

            if order_id != market_orders[0]:
                # All match-done events that happen between the market order is received and closed
                pass
            
            # Self-trade prevention events would also enter this loop, and would be counted as market filled.
            elif order_id == market_orders[0]: # Done market_order (filled)
                
                if aggregated_size != 0: # Self-trade prevention partially filled order also enter this loop
                
                    X_T[counter] = x_t.reshape((1, 40))
                    N_T[counter], past_event_time, past_event_side, past_event_type = extend_features(counter,
                                                                                                      past_event_time, 
                                                                                                      n_t,
                                                                                                      event_type,
                                                                                                      side,
                                                                                                      event_time,
                                                                                                      reason='market_filled',
                                                                                                      unopened=True)
                    counter += 1
                    
                else: # Self trade prevention without any match before canceling, no need to update
                    pass
                    
                    

        elif unopened_limit_orders:
            
            if order_id != unopened_limit_orders[0]: 
                # All match-done events that happen between the limit order order is received and closed
                pass
            
            # Self-trade prevention events would also enter this loop, and would be counted as limit filled.
            elif order_id == unopened_limit_orders[0]: # Done limit_order (filled)
                
                if aggregated_size != 0: # Self-trade prevention partially filled order also enter this loop
                
                    X_T[counter] = x_t.reshape((1, 40))
                    N_T[counter], past_event_time, past_event_side, past_event_type = extend_features(counter, 
                                                                                                      past_event_time, 
                                                                                                      n_t,
                                                                                                      event_type,
                                                                                                      side,
                                                                                                      event_time,
                                                                                                      reason='limit_filled',
                                                                                                      unopened=True)
                    counter += 1
                
                else: # Self trade prevention without any match before canceling, no need to update
                    pass
        
        else: # Normal path
            
            if past_event_time != event_time: # Different timestamp
                
                # Concatenate value to X_T
                X_T[counter] = x_t.reshape((1, 40))
                N_T[counter], past_event_time, past_event_side, past_event_type = extend_features(counter, 
                                                                                                  past_event_time, 
                                                                                                  n_t, 
                                                                                                  event_type,  
                                                                                                  side,
                                                                                                  event_time,
                                                                                                  price,
                                                                                                  size,
                                                                                                  reason=reason)
                counter += 1
                
            else: # Same timestamp
                
                counter -= 1

                # Concatenate value to X_T
                if past_event_side == side:
                    pass
                else:
                    both_sides = True
                X_T[counter] = x_t.reshape((1, 40))
                N_T[counter], past_event_time, past_event_side, past_event_type = extend_features(counter, 
                                                                                                  past_event_time,
                                                                                                  n_t, 
                                                                                                  event_type,
                                                                                                  side,
                                                                                                  event_time,
                                                                                                  price,
                                                                                                  size,
                                                                                                  reason=reason,
                                                                                                  continuous=True,
                                                                                                  both_sides=both_sides)
                counter += 1
# ----------------------------------------------------------------------------------------------------------------------#            
    
    elif event_type == 'open':

        if past_event_time != event_time:

            # Concatenate value to X_T
            X_T[counter] = x_t.reshape((1, 40))
            N_T[counter], past_event_time, past_event_side, past_event_type = extend_features(counter, 
                                                                                              past_event_time, 
                                                                                              n_t, 
                                                                                              event_type,  
                                                                                              side,
                                                                                              event_time,
                                                                                              price,
                                                                                              size)
            counter += 1

        else:
            
            # No done event followed by an open event is permitted, both comes from different order nature
            if event_type == past_event_type: 

                counter -= 1
                # Concatenate value to X_T
                if past_event_side == side:
                    pass
                else:
                    both_sides = True
                X_T[counter] = x_t.reshape((1, 40))
                N_T[counter], past_event_time, past_event_side, past_event_type = extend_features(counter, 
                                                                                                  past_event_time,
                                                                                                  n_t,
                                                                                                  event_type,
                                                                                                  side,
                                                                                                  event_time,
                                                                                                  price,
                                                                                                  size,
                                                                                                  continuous=True,
                                                                                                  both_sides=both_sides)
                counter += 1
                
            else:

                # Concatenate value to X_T
                X_T[counter] = x_t.reshape((1, 40))
                N_T[counter], past_event_time, past_event_side, past_event_type = extend_features(counter, 
                                                                                                  past_event_time, 
                                                                                                  n_t, 
                                                                                                  event_type,  
                                                                                                  side,
                                                                                                  event_time,
                                                                                                  price,
                                                                                                  size)
                counter += 1

# ----------------------------------------------------------------------------------------------------------------------# 
    elif event_type == 'change':
        
        # Concatenate value to X_T
        X_T[counter] = x_t.reshape((1, 40))
        N_T[counter], past_event_time, past_event_side, past_event_type = extend_features(counter, 
                                                                                          past_event_time,
                                                                                          n_t,
                                                                                          event_type,
                                                                                          side,
                                                                                          event_time,
                                                                                          price,
                                                                                          size)
        counter += 1
        
    return X_T, N_T, counter, past_event_time, past_event_side, past_event_type

### INITIALIZE CONTAINER FOR x_t, n_t, X_T, N_T and time

In [ ]:
if data_status != 'Missing':
    
    # Initialize x_t and n_t
    x_t, n_t = np.zeros((10, 4)), np.zeros((10, 2))
    x_t, n_t = update_x_t('sell', N, x_t, n_t)
    x_t, n_t = update_x_t('buy', N, x_t, n_t)

    # Initialize X_T and N_T
    X_T = np.zeros((messages_data.shape[0], 40)) # For price and volume features
    N_T = np.zeros((messages_data.shape[0], 33)) # For extended features

    # Initialize time and past event details
    past_event_time = json.loads(Message_Book['Events'][start_index])['time']
    past_event_side = json.loads(Message_Book['Events'][start_index])['side']
    past_event_type = json.loads(Message_Book['Events'][start_index])['type']

### RECONSTRUCTING THE LIMIT ORDER BOOK

In [ ]:
if data_status == 'Missing':
    print('Nothing to be processed')
else:
    # Processing day:
    print('Processing day: ', date)
    # Initial time
    print('Total number of messages: ', "{:,}".format(messages_data['Events'].shape[0]))
    print('Data status: ', data_status)
    print('Start time: ', ctime(time() - 60**2))
    print()

    # Reconstructing the LOB
    counter=0
    time_init = time()

    # Market and limit orders list
    market_orders = []
    unopened_limit_orders = []


    for i, message in enumerate(messages_data['Events']):

        # Setting the event
        event = json.loads(message)
        event_sequence = event['sequence']

        # Event characteristics
        event_type = event['type']

        # @@@@@@@@@@@@ RECEIVED @@@@@@@@@@@@
        if event_type == 'received':

            if event['order_type'] == 'market':
                market_orders.append(event['order_id'])
                # Reset weighted average price
                aggregated_multiplication = 0
                aggregated_size = 0                
                
            elif event['order_type'] == 'limit':
                unopened_limit_orders.append(event['order_id'])
                # Reset weighted average price
                aggregated_multiplication = 0
                aggregated_size = 0
                
            else:
                pass

        # @@@@@@@@@@@@ OPEN @@@@@@@@@@@@@@@@
        elif event_type == 'open':

            # Event information
            order_id = event['order_id']
            price = event['price']
            size = event['remaining_size']
            side = event['side']
            event_time = event['time']

            # Remove the order from the limit_orders list
            if i == 0: # First order might be type: 'open', and will not be on 'unopened _limit_orders'
                pass
            else:
                unopened_limit_orders.remove(order_id)

            if side == 'buy':
                bids = add_order_to_orderbook(order_id, price, size, side, bids)

                if float(price) >= x_t[N - 1, 2]:

                    # Create time frame after order book update
                    x_t, n_t = update_x_t(side=side, depth=N, x_t=x_t, n_t=n_t)
                
                X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                        N_T,
                                                                                                        counter,
                                                                                                        past_event_time,
                                                                                                        past_event_side,
                                                                                                        past_event_type,
                                                                                                        n_t,
                                                                                                        event_type,
                                                                                                        side,
                                                                                                        event_time,
                                                                                                        price=price,
                                                                                                        size=size)


            elif side == 'sell':
                asks = add_order_to_orderbook(order_id, price, size, side, asks)

                if float(price) <= x_t[N - 1, 0]:

                    # Create time frame after order book update
                    x_t, n_t = update_x_t(side=side, depth=N, x_t=x_t, n_t=n_t)
                
                X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                        N_T,
                                                                                                        counter,
                                                                                                        past_event_time,
                                                                                                        past_event_side,
                                                                                                        past_event_type,
                                                                                                        n_t,
                                                                                                        event_type,
                                                                                                        side,
                                                                                                        event_time,
                                                                                                        price=price,
                                                                                                        size=size)

        # @@@@@@@@@@@@ DONE @@@@@@@@@@@@
        elif event_type == 'done':

            # Event Information
            order_id = event['order_id']
            side = event['side']
            event_time = event['time']
            reason = event['reason']

            try: # Some done events with reason: filled do not have a 'remaining_size' field, I think is market orders filled
                size = event['remaining_size']
            except KeyError:
                size = '0'


            if side == 'buy':
                
                
                # Removing the order from market_orders or from unopened_limit_orders
                if order_id in market_orders: # This can be moved outside of: if side == 'buy'
                    
                    X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                            N_T,
                                                                                                            counter,
                                                                                                            past_event_time,
                                                                                                            past_event_side,
                                                                                                            past_event_type,
                                                                                                            n_t,
                                                                                                            event_type,
                                                                                                            side,
                                                                                                            event_time,
                                                                                                            reason=reason)
                    
                    # Removing market order
                    market_orders.remove(order_id)

                elif order_id in unopened_limit_orders: # This can be moved outside of: elif side == 'buy'
                    
                    X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                            N_T,
                                                                                                            counter,
                                                                                                            past_event_time,
                                                                                                            past_event_side,
                                                                                                            past_event_type,
                                                                                                            n_t,
                                                                                                            event_type,
                                                                                                            side,
                                                                                                            event_time,
                                                                                                            reason=reason)
                    
                    # Removing limit order
                    unopened_limit_orders.remove(order_id)
                
                else: 
                    
                    # Get the price before removing it
                    price = get_price_done(side, event, bids)

                    # Remove the order from order book
                    bids = remove_from_orderbook(side, order_id, bids)

                    if price >= x_t[N - 1, 2]:

                        # Create time frame after order book update
                        x_t, n_t = update_x_t(side=side, depth=N, x_t=x_t, n_t=n_t)

                    X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                            N_T,
                                                                                                            counter,
                                                                                                            past_event_time,
                                                                                                            past_event_side,
                                                                                                            past_event_type,
                                                                                                            n_t,
                                                                                                            event_type,
                                                                                                            side,
                                                                                                            event_time,
                                                                                                            price=price,
                                                                                                            size=size,
                                                                                                            reason=reason)


                        
            elif side == 'sell':

                # Removing the order from market_orders or from unopened_limit_orders
                if order_id in market_orders: # This can be moved outside of: if side == 'buy'
                    
                    X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                            N_T,
                                                                                                            counter,
                                                                                                            past_event_time,
                                                                                                            past_event_side,
                                                                                                            past_event_type,
                                                                                                            n_t,
                                                                                                            event_type,
                                                                                                            side,
                                                                                                            event_time,
                                                                                                            reason=reason)
                    # Removing market order
                    market_orders.remove(order_id)

                elif order_id in unopened_limit_orders: # This can be moved outside of: elif side == 'buy'
                    
                    X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                            N_T,
                                                                                                            counter,
                                                                                                            past_event_time,
                                                                                                            past_event_side,
                                                                                                            past_event_type,
                                                                                                            n_t,
                                                                                                            event_type,
                                                                                                            side,
                                                                                                            event_time,
                                                                                                            reason=reason)
                    # Removing limit order
                    unopened_limit_orders.remove(order_id)

                else:
                    # Get the price before removing it
                    price = get_price_done(side, event, asks)

                    # Remove the order from order book
                    asks = remove_from_orderbook(side, order_id, asks)

                    if price <= x_t[N - 1, 0]:

                        # Create time frame after order book update
                        x_t, n_t = update_x_t(side=side, depth=N, x_t=x_t, n_t=n_t)

                    X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                            N_T,
                                                                                                            counter,
                                                                                                            past_event_time,
                                                                                                            past_event_side,
                                                                                                            past_event_type,
                                                                                                            n_t,
                                                                                                            event_type,
                                                                                                            side,
                                                                                                            event_time,
                                                                                                            price=price,
                                                                                                            size=size,
                                                                                                            reason=reason)


        # @@@@@@@@@@@@ MATCH @@@@@@@@@@@@
        elif event_type == 'match':

            # Event Information
            maker_order_id = event['maker_order_id']
            taker_order_id = event['taker_order_id']
            size = event['size']
            side = event['side']
            price = event['price']
            event_time = event['time']

            if side == 'buy':
                # Subtract from orderbook
                bids = subtract_from_orderbook(maker_order_id, size, side, bids)
                
                # Calculating the average weighted price
                aggregated_multiplication += float(size) * float(price)
                aggregated_size += float(size)
                
                if float(price) >= x_t[N - 1, 2]:

                    # Create time frame after order book update
                    x_t, n_t = update_x_t(side=side, depth=N, x_t=x_t, n_t=n_t)

            elif side == 'sell':
                # Subtract from orderbook
                asks = subtract_from_orderbook(maker_order_id, size, side, asks)
                
                # Calculating the average weighted price
                aggregated_multiplication += float(size) * float(price)
                aggregated_size += float(size)
                
                if float(price) <= x_t[N - 1, 0]:

                    # Create time frame after order book update
                    x_t, n_t = update_x_t(side=side, depth=N, x_t=x_t, n_t=n_t)
                
        # @@@@@@@@@@@@ CHANGE @@@@@@@@@@@@
        elif event_type == 'change':

            # Event Information
            order_id = event['order_id']
            side = event['side']
            price = event['price']
            event_time = event['time']

            if price == '':
                pass

            else:

                size = event['new_size'] # Not all change orders have a 'new_size' field, 
                                         # Market orders have price = '', and no 'new_size' field.

                if side == 'buy':

                    if order_id in unopened_limit_orders: # Change messages for received but not yet open orders 
                        pass                              # can be ignored when building a real-time order book                                      

                    else:

                        # Change order size
                        bids = change_order_size(order_id, size, side, bids)

                        if float(price) >= x_t[N - 1, 2]:

                            # Create time frame after order book update
                            x_t, n_t = update_x_t(side=side, depth=N, x_t=x_t, n_t=n_t)

                        X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                                N_T,
                                                                                                                counter,
                                                                                                                past_event_time,
                                                                                                                past_event_side,
                                                                                                                past_event_type,
                                                                                                                n_t,
                                                                                                                event_type,
                                                                                                                side,
                                                                                                                event_time,
                                                                                                                price=price,
                                                                                                                size=size)


                elif side == 'sell':

                    if order_id in unopened_limit_orders: # Change messages for received but not yet open orders 
                        pass                              # can be ignored when building a real-time order book                                      

                    else:

                        # Change order size
                        asks = change_order_size(order_id, size, side, asks)

                        if float(price) <= x_t[N - 1, 0]:

                            # Create time frame after order book update
                            x_t, n_t = update_x_t(side=side, depth=N, x_t=x_t, n_t=n_t)

                        X_T, N_T, counter, past_event_time, past_event_side, past_event_type = aggregate_events(X_T,
                                                                                                                N_T,
                                                                                                                counter,
                                                                                                                past_event_time,
                                                                                                                past_event_side,
                                                                                                                past_event_type,
                                                                                                                n_t,
                                                                                                                event_type,
                                                                                                                side,
                                                                                                                event_time,
                                                                                                                price=price,
                                                                                                                size=size)


        # @@@@@@@@@@@@ ACTIVATE @@@@@@@@@@@@
        elif event_type == 'activate': 
            # Do nothing, this event_type is used for stop orders, after triggered it goes to the normal lyfecycle
            pass


        if i % 100000 == 0:

            print('Index: ', i, '  |  ', 'Sequence: ', "{:,}".format(event_sequence), '  |  ', 
                  'Elapsed time: ', '%.2f' % (time() - time_init), 's')
            time_init = time()

    # End time
    print()
    print('End time: ', ctime(time() - 60**2))
    print()

    # Truncate the numpy arrays
    X_T = X_T[ : counter]
    N_T = N_T[ : counter]

    # Save the data in the hdf5 file
    dataset = h5py.File(f'LOB Reconstructed/LOB_{date}_{hour}_{data_status}.hdf5', 'w')
    # Create the event records dataset and the order_id dataset
    dataset.create_dataset('data', data=X_T, dtype='f4')
    dataset.create_dataset('data_extend', data=N_T, dtype='f4')

    # Closing the hdf5 file
    dataset.close()

    # Verify that bids and asks matches
    if data_status == 'OK':
        checker()